# NDEx2 Client Objects

#### Modules Required for NDEx2 Client Tutorial

In [ ]:
import ndex2.client as nc
import io
import json
from IPython.display import HTML
#from time import sleep

#### access an NDEx server anonymously

In [ ]:
anon_ndex=nc.Ndex2("http://public.ndexbio.org")

#### A client object using a specific user account

In [ ]:
my_account="my account"
my_password="my password"

#### update_status()

In [ ]:
if my_account == "my account":
    print('Please enter your username before proceeding')
    my_account = input()
if my_password=="my password":
    print('Please enter your password before proceeding')
    my_password = input()

try:
    my_ndex=nc.Ndex2("http://public.ndexbio.org", my_account, my_password)
    my_ndex.update_status()
    networks = my_ndex.status.get("networkCount")
    users = my_ndex.status.get("userCount")
    groups = my_ndex.status.get("groupCount")
    print("my_ndex client: %s networks, %s users, %s groups" % (networks, users, groups))
except Exception as inst:
    print("Could not access account %s with password %s" % (my_account, my_password))
    print(inst.args)

#### get_user_by_username(username)

In [ ]:
my_ndex.get_user_by_username("Jeffreyc")

#### find_networks(search_string=”“, account_name=None, start=0, size=100)

In [ ]:
#This method is deprecated; search_networks should be used instead.

#### search_networks(search_string=”“, account_name=None, start=0, size=100, include_groups=False)

In [ ]:
list_tutorial_networks=my_ndex.search_networks('tutorial')
print("%s netwoks found." % (len(list_tutorial_networks)))
print ("\nNetwork names and uuids:\n")
for i in list_tutorial_networks.get('networks'): 
    print( "%s -->> %s" % (i.get('name'), i.get('externalId')))

#### get_network_summary(network_id)

In [ ]:
ns=my_ndex.get_network_summary('c9243cce-2d32-11e8-b939-0ac135e8bacf')
def summary2table(object):
    table = "<table>"
    for key, value in object.items():
        if key == "warnings":
            warning_list = ""
            for warning in value:
                warning_list += "%s<br>" % warning
            value = warning_list
        if key == "properties":
            property_table = "<table>"
            for property in value:
                property_table += "<tr>"
                property_table += "<td>%s</td><td>%s</td>" % (property.get("predicateString"), property.get("value"))
                property_table += "</tr>"
            property_table += "</table>"
            value = property_table
        table += "<tr>"
        table += "<td>%s</td><td>%s</td>" % (key, value)
        table += "</tr>"
    table += "</table>"
    return table

HTML(summary2table(ns))

#### get_network_as_cx_stream(network_id)

In [ ]:
response=my_ndex.get_network_as_cx_stream('c9243cce-2d32-11e8-b939-0ac135e8bacf')
raw_cx = response.json()


#the code below copied for the v2.0 Tutorial. I don't understand well
print("Received %s characters of CX" % len(response.content))
def getNumberOfNodesAndEdgesFromCX(cx):
    numberOfEdges = numberOfNodes = 0;
    for aspect in cx:
        #print('key')
        if 'nodes' in aspect:
            numberOfNodes = len(aspect['nodes'])
        if 'metaData' in aspect:
            metaData = aspect['metaData']
            for element in metaData:
                if (('name' in element) and (element['name'] == 'nodes')):
                    numberOfNodes = element.get('elementCount')
                if (('name' in element) and (element['name'] == 'edges')):
                    numberOfEdges = element.get('elementCount')
            break
    return numberOfNodes, numberOfEdges

nodes, edges = getNumberOfNodesAndEdgesFromCX(raw_cx)
print("network contains %s nodes and %s edges." % (nodes, edges))

#### get_neighborhood_as_cx_stream(network_id, search_string, search_depth=1, edge_limit=2500)

In [ ]:
#return a Response object or a CX stream?
query_result_cx_stream = my_ndex.get_neighborhood_as_cx_stream('c9243cce-2d32-11e8-b939-0ac135e8bacf','XRN1')
                                       
print (query_result_cx_stream)

#### save_cx_stream_as_new_network(cx_stream)

In [ ]:
#I tried the code below, but got an error message:'Response' object has no attribute 'encode'
uri_subset_cx_stream = my_ndex.save_cx_stream_as_new_network(query_result_cx_stream)
uuid_subset_cx_stream = uri_subset_cx_stream.rpartition('/')[-1]
print ("URI of the subset is: ", uri_subset_cx_stream)
print ("UUID of the subset is: ", uuid_subset_cx_stream)

#### get_neighborhood(network_id, search_string, search_depth=1, edge_limit=2500)

In [ ]:
#return a CX JSON object
query_result_cx=my_ndex.get_neighborhood('c9243cce-2d32-11e8-b939-0ac135e8bacf','XRN1')
                                       
for aspect in query_result_cx:
    if 'nodes' in aspect:
        number_of_nodes = len(aspect['nodes'])
    if 'edges' in aspect:
        number_of_edges = len(aspect['edges'])

print("Query result network contains %s nodes and %s edges." % (number_of_nodes, number_of_edges))

#### save_new_network(cx)

In [ ]:
uri_subset = my_ndex.save_new_network(query_result_cx)
uuid_subset = uri_subset.rpartition('/')[-1]
print ("URI of the subset is: ", uri_subset)
print ("UUID of the subset is: ", uuid_subset)

new_summary = my_ndex.get_network_summary(uuid_subset)
 
if new_summary.get("isValid"):
        print("New network has been validated by the server.\n")
else:
    print("failed\n")
    print (new_summary)

#### update_cx_network(cx_stream, network_id)

In [ ]:
#did not try this method since don't understand well

#### get_network_ids_for_user(account_name)

In [ ]:
list_uuid = my_ndex.get_network_ids_for_user("Jeffreyc")
print (list_uuid)

#### delete_network(network_id)

In [ ]:
indexval = 0
for i in range(len(list_uuid)):
    if (list_uuid[indexval] != uuid_subset):
        my_ndex.delete_network(list_uuid[indexval])        
    else:
        print("URI of the newly created network %s is %s" % (uuid_subset, uri_subset))
    indexval += 1

###### set_network_system_properties(network_id, network_system_properties)

##### make_network_private(network_id)

#### make_network_public(network_id)

In [ ]:
my_ndex.make_network_public(uuid_subset)

#### set_read_only(network_id, value)

In [ ]:
#revert network to read-write 
my_ndex.set_read_only(uuid_subset, False)
new_summary = my_ndex.get_network_summary(uuid_subset)
print("The read only status has been reverted to %s" % new_summary.get('isReadOnly'))

##### update_network_group_permission(group_id, network_id, permission)

##### grant_networks_to_group(group_id, network_ids, permission=”READ”)

##### update_network_user_permission(user_id, network_id, permission)

##### grant_network_to_user_by_username(username, network_id, permission)

##### grant_networks_to_user(user_id, network_ids, permission=”READ”)

#### update_network_profile(network_id, network_profile)

In [ ]:
network_profile={"name":"My Network", "description":"learn from Tutorial", "version":"1.0"}
my_ndex.update_network_profile(uuid_subset, network_profile)    

##### set_network_properties(network_id, network_properties)

##### get_provenance(network_id)

##### set_provenance(network_id, provenance)

##### get_network_summaries_for_user(account_name)

##### get_task_by_id(task_id)

# NiceCX Objects

#### Modules Required for NiceCX Tutorial

In [24]:
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os

#### set_name()

In [25]:
#create/initial an empty niceCX network before setting the network name
NiceCX_creatures = NiceCXNetwork()
NiceCX_creatures.set_name("new web")

#### ndex2.create_nice_cx_from_raw_cx(cx)

In [ ]:
NiceCX_from_stream = ndex2.create_nice_cx_from_raw_cx(raw_cx)
NiceCX_from_stream.print_summary()

##### ndex2.create_nice_cx_from_file(path)

##### ndex2.create_nice_cx_from_networkx(G)

##### ndex2.create_nice_cx_from_pandas(df)

#### ndex2.create_nice_cx_from_server(server, username=None, password=None, uuid=None)

In [26]:
uuid = "1c69beff-1229-11e6-a039-06603eb7f303"
nice_cx_network = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org', uuid=uuid)
nice_cx_network.print_summary()

Name: ChEMBL - High Affinity Compounds vs human targets (Commercially available)
Nodes: 3564
Edges: 11564
Node Attributes: 13569
Edge Attributes: 57820



#### get_nodes()

In [27]:
node_name_list = []
node_id_list =[]

for node_id, node in nice_cx_network.get_nodes():
    node_name = node.get('n')  
    #node_name = list(node.values())[1]
    node_name_list.append(node_name)
    node_id_list.append(node_id)
    
name_id_map = dict(zip(node_name_list,node_id_list))

#name_id_map = {node.get('n'): node.get('@id') for node_id, node in nice_cx_network.get_nodes()}



#### get_node(node)

In [28]:
#sample_node_id = name_id_map.get('ACPP')
sample_node_id = name_id_map['ACPP']
sample_node = nice_cx_network.get_node( sample_node_id )
print(sample_node)

{'@id': 87, 'n': 'ACPP', 'r': 'uniprot:P15309'}


#### get_node_attributes(node)

In [29]:
nice_cx_network.get_node_attributes(sample_node)

[{'po': 87, 'n': 'Target Name', 'v': 'Prostatic acid phosphatase'},
 {'po': 87,
  'n': 'BindingDB Target Page',
  'v': 'http://www.bindingdb.org/jsp/dbsearch/PrimarySearch_ki.jsp?energyterm=kJ/mole&tag=pol&polymerid=50000972&target=Prostatic+acid+phosphatase&column=ki&startPg=0&Increment=50&submit=Search'},
 {'po': 87, 'n': 'type', 'v': 'protein'}]

#### get_node_attribute(node, attribute_name, subnetwork=None)

In [30]:
attr_dict = nice_cx_network.get_node_attribute(sample_node_id, 'type')

attr_value = attr_dict.get('v')

print (attr_dict)
print(attr_value)


{'po': 87, 'n': 'type', 'v': 'protein'}
protein


##### get_node_attribute_objects(node, attribute_name)

In [12]:
#get_node_attribute_objects() is deprecated.  Please use get_node_attribute() instead

#### create_node(name, represents=None)

In [31]:
new_network_node_id_list =[]

for i in range (len(node_name_list)):
    new_node_id = NiceCX_creatures.create_node(node_name_list[i]) 
    new_network_node_id_list.append(new_node_id)
    
#print (new_network_node_id_list)

#### add_node(node)

In [ ]:
#This method has been deprecated. Please use create_node()

#### set_node_attribute(node, attribute_name, values, type=None, subnetwork=None)

In [32]:
node_id = new_network_node_id_list[0]
node_attr_name = "type"
node_attr_value = attr_value

NiceCX_creatures.set_node_attribute(node_id, node_attr_name, node_attr_value) 

NiceCX_creatures.get_node_attribute(node_id, node_attr_name)

{'po': 0, 'n': 'type', 'v': 'protein'}

#### get_edges()

In [45]:
edge_s_list = []
edge_t_list = []
edge_i_list = []

for edge_id, edge in nice_cx_network.get_edges():        
    edge_s = edge.get('s') 
    edge_t = edge.get('t') 
    edge_i = edge.get('i') 
    
    edge_s_list.append(edge_s)
    edge_t_list.append(edge_t)
    edge_i_list.append(edge_i)
    
print (edge_id)

11563
11564


#### get_edge_attributes(edge)

In [38]:
sample_edge_id = edge_id
nice_cx_network.get_edge_attributes(sample_edge_id)

[{'po': 11563, 'n': 'Affinity (nM)', 'v': '>10.0'},
 {'po': 11563,
  'n': 'BindingDB Details of Measurement',
  'v': 'http://www.bindingdb.org/jsp/dbsearch/Summary_ki.jsp?reactant_set_id=51082255'},
 {'po': 11563,
  'n': 'All BindingDB Data for Ligand-Target Pair',
  'v': 'http://www.bindingdb.org/jsp/dbsearch/PrimarySearch_ki.jsp?energyterm=kJ/mole&tag=r21&monomerid=50442760&enzyme=Tyrosine-protein+kinase+ZAP-70&column=ki&startPg=0&Increment=50&submit=Search'},
 {'po': 11563, 'n': 'Data Curation Source', 'v': 'ChEMBL'},
 {'po': 11563,
  'n': 'citation',
  'v': ['pubmed:24100158'],
  'd': 'list_of_string'}]

#### get_edge_attribute(edge, attribute_name, subnetwork=None)

In [40]:
edge_attr_list = nice_cx_network.get_edge_attributes(sample_edge_id)
sample_edge_attr_name = edge_attr_list[0].get('n')
sample_edge_attr_value = edge_attr_list[0].get('v')

nice_cx_network.get_edge_attribute(sample_edge_id, sample_edge_attr_name)

Affinity (nM)
>10.0


{'po': 11563, 'n': 'Affinity (nM)', 'v': '>10.0'}

#### get_edge_attribute_objects(edge, attribute_name)

In [ ]:
#This method has been deprecated. Please use get_edge_attributes()

#### create_edge(source, target, interaction)

In [47]:
new_edge_id_list = []

for i in range(len(edge_s_list)):
    new_edge_id = NiceCX_creatures.create_edge(edge_s_list[i], edge_t_list[i], edge_i_list[i])
    new_edge_id_list.append(new_edge_id)           

11564


#### set_edge_attribute(edge, attribute_name, values, type=None, subnetwork=None)

In [48]:
NiceCX_creatures.set_edge_attribute(new_edge_id_list[0], sample_edge_attr_name, sample_edge_attr_value )

#### add_edge(edge)

In [ ]:
#This method has been deprecated. Please use create_edge()

get_name()

set_name(network_name)

getSummary()

set_network_attribute(name=None, values=None, type=None, subnetwork_id=None)

get_network_attribute(attribute_name, subnetwork_id=None)

get_network_attribute_objects(attribute_name)

get_network_attributes()

get_metadata()

set_metadata()

getProvenance()

set_provenance(provenance)

get_context(context)

set_context()

set_context()

set_context()

get_opaque_aspect(aspect_name)

set_opaque_aspect(aspect_name, aspect_elements)

set_opaque_aspect(aspect_name, aspect_elements)

get_opaque_aspect_names()

to_cx()

to_cx_stream()

#### to_networkx()

In [ ]:
networkx_network = nice_cx_network.to_networkx()

#### to_pandas_dataframe()

In [ ]:
pandas_network = nice_cx_network.to_pandas_dataframe()

upload(ndex_server, username, password, update_uuid=None)

apply_template(server, username, password, uuid)

addNode(json_obj=None)

add_edge_element(json_obj=None, edge)

addNetworkAttribute(json_obj=None)